In [ ]:
"""
1. 사용 데이터셋 : 구글드라이브 files 폴더 내 파일

2. 데이터셋 확인 후
 -> 지금까지 배운 내용들을 기반으로
 -> 1개 이상의 주제 만들어서 수행
 -> 주제별로 ipynb 파일 각각 사용
 
3. 개별 제출
 -> 제출위치 : 구글드라이브 > 개별실습(제출)
 -> 압축하여 03_(실습)_본인이름.zip 으로 제출
"""

### 라이브러리 정의

In [1]:
import pandas as pd
import numpy as np
import re

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Dropout, SimpleRNN, Embedding, LSTM, GRU, RepeatVector, TimeDistributed, Bidirectional
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
tf.keras.utils.set_random_seed(42)

import matplotlib.pyplot as plt

In [2]:
Exe_QnA_Data = pd.read_csv("./files/04_Exe_QnA_Data.csv")
Exe_QnA_Data.info()
Exe_QnA_Data.head(2)

### 데이터 요약
# - 결측값은 없음
# - 데이터로 보아 일반 챗봇 또는 label을 활용하여 일상다반사0/이별(부정)1/사랑(긍정)2 비율을 나타내 줄 수도 있어보임
# 머릿 속 예시)
# 입력자 : 입력값
# 보키봇 : 챗봇의 답변
# 보키봇 : 사용자님의 글은 [비율]%의 확률로 [일상다반사0/이별(부정)1/사랑(긍정)2] 입니다. 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Q                              11823 non-null  object
 1   A                              11823 non-null  object
 2   label(일상다반사0/이별(부정)1/사랑(긍정)2)  11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


,Q,A,label(일상다반사0/이별(부정)1/사랑(긍정)2)
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0


In [3]:
### 컬럼명 간단화 작업
# - 일상다반사0/이별(부정)1/사랑(긍정)2
Exe_QnA_Data = Exe_QnA_Data.rename(columns={"label(일상다반사0/이별(부정)1/사랑(긍정)2)": "label"})
Exe_QnA_Data.head(1)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0


In [4]:
### label 컬럼의 비율 확인
Exe_QnA_Data["label"].value_counts()

label
0    5290
1    3570
2    2963
Name: count, dtype: int64

In [5]:
### Q & A 중복 값 확인
print(Exe_QnA_Data["Q"].duplicated().value_counts())
print(Exe_QnA_Data["A"].duplicated().value_counts())

duplicated_QA = Exe_QnA_Data.duplicated(subset=["Q", "A"], keep=False)

duplicates = Exe_QnA_Data[duplicated_QA]

duplicates_sorted = duplicates.sort_values(by=["Q", "A"]).reset_index()
duplicates_sorted.head(10)

Q
False    11662
True       161
Name: count, dtype: int64
A
False    7779
True     4044
Name: count, dtype: int64


,index,Q,A,label
0,152,결혼이나 하지 왜 자꾸 나한테 화 내냐구!,힘들겠네요.,0
1,5527,결혼이나 하지 왜 자꾸 나한테 화 내냐구!,힘들겠네요.,1
2,189,고백하고 후회하면 어떡하지,후회는 후회를 낳을뿐이에요. 용기 내세요.,0
3,5537,고백하고 후회하면 어떡하지,후회는 후회를 낳을뿐이에요. 용기 내세요.,1
4,226,공부는 내 체질이 아닌 것 같아,확신이 없나봐요.,0
5,5542,공부는 내 체질이 아닌 것 같아,확신이 없나봐요.,1
6,377,기숙사 괜찮을까,혼자 사는 것보다 불편하겠죠.,0
7,5704,기숙사 괜찮을까,혼자 사는 것보다 불편하겠죠.,1
8,592,나는 좋은데 ….,현실의 벽에 부딪혔나봐요.,0
9,5774,나는 좋은데 ….,현실의 벽에 부딪혔나봐요.,1


In [6]:
### 중복값 제거
# 같은 Q+A인데 label이 서로 다른 경우만 추출
dup = Exe_QnA_Data[Exe_QnA_Data.duplicated(subset=["Q", "A"], keep=False)]
conflict = dup.groupby(["Q", "A"]).filter(lambda x: x["label"].nunique() > 1)

# 충돌 데이터 제거
Exe_QnA_Data = Exe_QnA_Data.drop(index=conflict.index)

# Q + A + label이 모두 동일한 값도 제거 (혹시 몰라서 해주기...)
Exe_QnA_Data = Exe_QnA_Data.drop_duplicates(subset=["Q", "A", "label"])

In [7]:
# 질문과 답변 리스트
questions = list(Exe_QnA_Data["Q"].astype(str))
answers = list(Exe_QnA_Data["A"].astype(str))

print(len(questions), len(answers))

11677 11677


In [8]:
### 텍스트 문장 데이터를 토큰화
# oov_token="<OOV>"는 훈련 때 없던 새로운 단어를 모델이 인식할 수 있도록 특별 토큰으로 바꿔줘서, 에러 없이 안정적인 예측이 가능하게 해줍니다.
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer

### 텍스트에서 단어 추출 및 인덱스화
tokenizer.fit_on_texts(questions+answers)

### 전체 분리된 단어들의 갯수 확인하기
len(tokenizer.word_index)

### 단어들의 순서 확인하기
tokenizer.word_index

{'<OOV>': 1,
 '거예요': 2,
 '수': 3,
 '너무': 4,
 '더': 5,
 '좋아하는': 6,
 '거': 7,
 '잘': 8,
 '것': 9,
 '많이': 10,
 '안': 11,
 '좋은': 12,
 '게': 13,
 '사람': 14,
 '좀': 15,
 '같아요': 16,
 '있어요': 17,
 '싶어': 18,
 '있을': 19,
 '같아': 20,
 '나': 21,
 '사람이': 22,
 '마세요': 23,
 '건': 24,
 '내가': 25,
 '이제': 26,
 '마음이': 27,
 '내': 28,
 '해보세요': 29,
 '썸': 30,
 '다': 31,
 '그': 32,
 '다른': 33,
 '어떻게': 34,
 '왜': 35,
 '있는': 36,
 '이별': 37,
 '싶다': 38,
 '다시': 39,
 '시간이': 40,
 '수도': 41,
 '정말': 42,
 '오늘': 43,
 '또': 44,
 '좋을': 45,
 '하고': 46,
 '하는': 47,
 '없어': 48,
 '것도': 49,
 '같이': 50,
 '있어': 51,
 '할': 52,
 '될': 53,
 '없어요': 54,
 '자꾸': 55,
 '바랄게요': 56,
 '한': 57,
 '걸': 58,
 '돼요': 59,
 '제가': 60,
 '진짜': 61,
 '먼저': 62,
 '저도': 63,
 '해': 64,
 '헤어진지': 65,
 '그런': 66,
 '나를': 67,
 '있을까': 68,
 '좋아요': 69,
 '보세요': 70,
 '마음을': 71,
 '여자친구가': 72,
 '계속': 73,
 '될까': 74,
 '없는': 75,
 '남자친구가': 76,
 '못': 77,
 '드세요': 78,
 '먹고': 79,
 '때': 80,
 '하세요': 81,
 '마음': 82,
 '좋죠': 83,
 '않아요': 84,
 '힘든': 85,
 '바랍니다': 86,
 '그럴': 87,
 '연락': 88,
 '일이': 89,
 '연애': 90,
 '해요': 

In [10]:
### 훈련시 사용할 말뭉치 갯수(단어 집합 크기)
vocab_size = len(tokenizer.word_index) + 1
vocab_size

20646

In [11]:
### 질문 데이터 인덱스화 하기
questions_sequences = tokenizer.texts_to_sequences(questions)

print(len(questions_sequences))
print(questions_sequences)

### 질문에 대한 최대값, 최소값, 중앙값 확인하기
questions_length = np.array([len(i) for i in questions_sequences])

print(np.max(questions_length))
print(np.min(questions_length))
print(np.median(questions_length))

q_padded = pad_sequences(questions_sequences, padding="post")
q_padded

11677
[[9419, 9420], [9421, 574, 965], [5120, 1694, 38], [5120, 823, 1694, 38], [9422, 5121], [5122, 9423], [5122, 146], [966, 9424, 35, 9425], [966, 9426, 7, 1084, 260, 47, 195], [966, 9427, 55, 9428], [9429, 123, 1961, 31, 2865], [386, 575], [386, 3792, 575], [1320, 5123, 716], [9430, 9431, 9432], [9433, 639, 5124], [9434, 9435, 2866], [5125, 9436, 2867], [5125, 9437, 5126, 20], [3793, 4, 10, 9438], [3793, 5127, 418, 9439], [3793, 1321, 2370], [226, 464, 24, 275], [2371, 283, 2868, 387], [2371, 283, 2372], [2371, 283, 1962, 967], [2371, 51], [5128, 3794, 1085], [5129, 9440], [9441, 105, 18], [2869, 9442], [2869, 9443], [2869, 640, 967], [2869, 283, 9444], [9445, 606], [1493, 5130], [9446, 283, 2870], [2871, 263, 9447], [9448], [9449, 576, 2872, 48], [2873, 9450, 3795], [2873, 1695, 195], [2873, 5131, 195], [3796, 94, 887], [3796, 512], [9451, 5132], [9452, 168], [717, 1086, 388, 155], [717, 1086], [5133, 5134, 79, 38], [5133, 5134, 389], [418, 236], [418, 2874, 9, 20], [418, 3797, 51

array([[9419, 9420,    0, ...,    0,    0,    0],
       [9421,  574,  965, ...,    0,    0,    0],
       [5120, 1694,   38, ...,    0,    0,    0],
       ...,
       [   1, 1437,  190, ...,    0,    0,    0],
       [  85,   90,   12, ...,    0,    0,    0],
       [1028,    1,    0, ...,    0,    0,    0]])

In [12]:
### 답변 데이터 인덱스화 하기
answers_sequences = tokenizer.texts_to_sequences(answers)

print(len(answers_sequences))
print(answers_sequences)

### 답변에 대한 최대값, 최소값, 중앙값 확인하기
answers_length = np.array([len(i) for i in answers_sequences])

print(np.max(answers_length))
print(np.min(answers_length))
print(np.median(answers_length))

a_padded = pad_sequences(answers_sequences, padding="post")
a_padded

11677
[[2152, 44, 3378], [1055, 3379], [2729, 457, 83], [2729, 457, 83], [7084, 7085], [39, 1121, 343, 13, 82, 7086], [39, 1121, 343, 13, 82, 7086], [8, 981, 19, 41, 17], [214, 4650, 29], [214, 4650, 29], [7087, 7088], [32, 305, 87, 2], [32, 305, 87, 2], [7089, 1448], [2444, 39, 7090, 2], [3380, 7091], [1058, 642, 4356, 81], [312, 230, 7092, 7093, 4651], [312, 230, 7092, 7093, 4651], [1289, 7094, 5, 7095], [1290, 336], [1289, 7094, 5, 7095], [226, 464, 534, 3381, 2632, 3382, 1088, 3383, 214, 2629, 23], [1002, 1493, 363, 157, 516, 402, 187], [1002, 1493, 363, 157, 516, 402, 187], [1002, 1493, 363, 157, 516, 402, 187], [594, 680, 1134, 119, 623, 270, 2221, 428, 119, 1103, 565, 17], [594, 680, 1134, 119, 623, 270, 2221, 428, 119, 1103, 565, 17], [5, 4652, 1171, 670], [872], [986, 2704, 4653, 40, 7096], [986, 2704, 4653, 40, 7096], [1002, 1493, 363, 157, 516, 402, 187], [12, 1172], [5, 4652, 1171, 670], [594, 680, 1134, 119, 623, 270, 2221, 428, 119, 1103, 565, 17], [12, 1172], [42, 707, 1

array([[2152,   44, 3378, ...,    0,    0,    0],
       [1055, 3379,    0, ...,    0,    0,    0],
       [2729,  457,   83, ...,    0,    0,    0],
       ...,
       [   1,    0,    0, ...,    0,    0,    0],
       [   8, 1853,    3, ...,    0,    0,    0],
       [9138, 1394,  107, ...,    0,    0,    0]])

In [13]:
# 라벨 one-hot encoding
labels = Exe_QnA_Data["label"].astype(int)
labels_cat = to_categorical(labels, num_classes=3)
labels_cat

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]], dtype=float32)

In [14]:
# 훈련/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(q_padded, labels_cat, test_size=0.2, random_state=42)
X_train_ans, X_val_ans = train_test_split(a_padded, test_size=0.2, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_train_ans.shape, X_val_ans.shape)

(9341, 15) (9341, 3)
(2336, 15) (2336, 3)
(9341, 21) (2336, 21)


### 감정 분류 모델 정의

In [16]:
from keras.layers import LayerNormalization

### 감정 분류 모델 정의
emotion_model = Sequential([
    # 입력계층
    Embedding(
        input_dim=vocab_size,
        output_dim=64,
        input_length=X_train.shape[1]
    ),
    # 은닉계층
    Bidirectional(
        LSTM(
            units=64,
            return_sequences=True
        )
    ),
    LayerNormalization(),
    Bidirectional(
        LSTM(
            units = 32
        )
    ),
    Dropout(0.5),
    # 출력계층
    Dense(
        units=3,
        activation="softmax"
    )

])

emotion_model.summary()

emotion_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss ="categorical_crossentropy",
    metrics = ["accuracy"]
)

mc = ModelCheckpoint(
    "./model/best_emotion_model.keras",
    save_best_only=True
)

# es = EarlyStopping(
#     patience=5,
#     restore_best_weights=True
# )

emotion_history = emotion_model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=128,
    callbacks=[mc]
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 64)            1321344   
                                                                 
 bidirectional_2 (Bidirectio  (None, 15, 128)          66048     
 nal)                                                            
                                                                 
 layer_normalization_1 (Laye  (None, 15, 128)          256       
 rNormalization)                                                 
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                      

### Seq2Seq 모델 정의

In [17]:
# Seq2Seq 모델 정의
answer_model = Sequential([
    # 입력계층
    Embedding(
        input_dim=vocab_size,
        output_dim=64,
        input_length=X_train_ans.shape[1]
    ),
    # 은닉계층
    LSTM(
        units=64,
        activation="tanh",
    ),
    Dropout(0.2),
    RepeatVector(X_train_ans.shape[1]),
    # 은닉계층
    LSTM(
        units=64,
        activation="tanh",
        return_sequences=True
    ),
    # 출력계층
    TimeDistributed(
        Dense(
            units=vocab_size, 
            activation="softmax"
        )
    )
])

answer_model.summary()

answer_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss ="sparse_categorical_crossentropy",
    metrics = ["accuracy"]
)

mc = ModelCheckpoint(
    "./model/best_answer_model.keras",
    save_best_only=True
)

es = EarlyStopping(
    patience=5,
    restore_best_weights=True
)

answer_history = answer_model.fit(
    X_train_ans,
    X_train_ans,
    validation_data=(X_val_ans, X_val_ans),
    epochs=100,
    batch_size=128,
    callbacks=[mc, es]
)


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 21, 64)            1321344   
                                                                 
 lstm_4 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 repeat_vector (RepeatVector  (None, 21, 64)           0         
 )                                                               
                                                                 
 lstm_5 (LSTM)               (None, 21, 64)            33024     
                                                                 
 time_distributed (TimeDistr  (None, 21, 20646)        1341990   
 ibuted)                                              

In [1]:
best_emotion_model = keras.models.load_model("./model/best_emotion_model.keras")
best_answer_model = keras.models.load_model("./model/best_answer_model.keras")

emotion_train_score = best_emotion_model.evaluate(X_train, y_train)
emotion_val_score = best_emotion_model.evaluate(X_val, y_val)

answer_train_score = best_answer_model.evaluate(X_train_ans, np.expand_dims(X_train_ans, -1))
answer_val_score = best_answer_model.evaluate(X_val_ans, np.expand_dims(X_val_ans, -1))
print("\n\n")
print(f"감성 훈련 데이터 | 손실율 : {emotion_train_score[0]}, 정확도 : {emotion_train_score[1]}")
print(f"감성 검증 데이터 | 손실율 : {emotion_val_score[0]}, 정확도 : {emotion_val_score[1]}")
print("\n================================================================================\n")
print(f"답변 생성 훈련 데이터 | 손실율 : {answer_train_score[0]}, 정확도 : {answer_train_score[1]}")
print(f"답변 생성 검증 데이터 | 손실율 : {answer_val_score[0]}, 정확도 : {answer_val_score[1]}")


NameError: name 'keras' is not defined

In [ ]:
# 감정 예측 함수
def predict_emotion(text):
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=X_train.shape[1], padding='post')
    pred = emotion_model.predict(padded_seq, verbose=0)[0]
    label_map = {0: "일상다반사", 1: "이별(부정)", 2: "사랑(긍정)"}
    for i, prob in enumerate(pred):
        print(f"{label_map[i]}: {prob*100:.2f}%")
    print(f"> 최종 예측: {label_map[pred.argmax()]}")
    print("==============================")

# 답변 생성 함수
def generate_answer(text):
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=X_train_ans.shape[1], padding='post')
    pred = answer_model.predict(padded_seq, verbose=0)
    pred = np.argmax(pred[0], axis=-1)
    answer = ' '.join([tokenizer.index_word.get(i, '') for i in pred if i != 0])
    return answer

# 사용자 입력에 대한 응답 생성
while True:
    user_input = input("질문을 입력하세요 (종료는 'Q'): ")
    if user_input.strip().upper() == "Q":
        print("보키봇 : 안녕히 가세요~")
        break
    print("사용자 :", user_input)
    answer = generate_answer(user_input)
    print("예상답변 :", answer)
    predict_emotion(user_input)

사용자 : 사랑해
예상답변 : <OOV> <OOV>
일상다반사: 0.00%
이별(부정): 0.00%
사랑(긍정): 100.00%
> 최종 예측: 사랑(긍정)
사용자 : 안녕 나는 너무 배고파
예상답변 : <OOV> <OOV>
일상다반사: 100.00%
이별(부정): 0.00%
사랑(긍정): 0.00%
> 최종 예측: 일상다반사
사용자 : 이런 너무 극단적이잖아
예상답변 : <OOV> <OOV>
일상다반사: 0.00%
이별(부정): 100.00%
사랑(긍정): 0.00%
> 최종 예측: 이별(부정)
사용자 : 
예상답변 : <OOV> <OOV>
일상다반사: 10.92%
이별(부정): 88.80%
사랑(긍정): 0.29%
> 최종 예측: 이별(부정)
사용자 : 
예상답변 : <OOV> <OOV>
일상다반사: 10.92%
이별(부정): 88.80%
사랑(긍정): 0.29%
> 최종 예측: 이별(부정)
보키봇 : 안녕히 가세요~
